<a href="https://colab.research.google.com/github/gyeongdong13/Study/blob/main/CNN_%EC%82%AC%EC%A0%84%ED%9B%88%EB%A0%A8%EB%AA%A8%EB%8D%B8_CIFAR10_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from tensorflow.keras.datasets import cifar10

# 전체 6만개 데이터 중, 5만개는 학습 데이터용, 1만개는 테스트 데이터용으로 분리
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
print(train_images.shape, train_labels.shape)
print(test_images.shape, test_labels.shape)

170498071/170498071 [==============================] - 11s 0us/step
(50000, 32, 32, 3) (50000, 1)
(10000, 32, 32, 3) (10000, 1)


In [ ]:
# 이미지 값 변환
train_images_2 = np.array(train_images/255.0, dtype=np.float32)
test_images_2 = np.array(test_images/255.0, dtype=np.float32)

# 라벨 값 변환
train_labels_2 = np.array(train_labels, dtype=np.float32)
test_labels_2 = np.array(test_labels, dtype=np.float32)

In [ ]:
train_labels_2 = train_labels.squeeze()
test_labels_2 = test_labels.squeeze()

# from tensorflow.keras.utils import to_categorical
# train_labels_2 = to_categorical(train_labels_2)
# test_labels_2 = to_categorical(test_labels_2)

In [ ]:
from sklearn.model_selection import train_test_split

train_x, val_x, train_y, val_y = train_test_split(train_images_2, train_labels_2, test_size=0.2, random_state=1)
train_x.shape, val_x.shape, train_y.shape, val_y.shape

((40000, 32, 32, 3), (10000, 32, 32, 3), (40000,), (10000,))

사전훈련 모델 불러오기

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.applications import ResNet50V2

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense , Conv2D , Dropout , Flatten , GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

In [ ]:
# include_top=False로 기존 imagenet용 classifier 층들을 다 제거
#weight는 전이학습을 위해 imagenet 학습된 weight를 초기 weight로 사용

base_model = VGG16(input_shape=(32, 32, 3), include_top=False, weights='imagenet') # 사전학습모델 불러오기(전이학습 이용)
# input_shape에 우리에 맞는 데이터 입력, include_top=False로 지정해서 윗부분은 고치지 않겠다는 의미(컨볼루션이랑 풀링은 제공된 그대로 사용하겠다는 의미)
# weights는 imagenet으로 학습된 데이터를 가중치로 가져온다는 것

# include_top을 False로 지정해서 여기서는 필요없는 부분
# 전이학습을 하기 위해, CNN층의 학습 차단
# for layer in base_model.layers:
#   layer.trainable = False

bm_output = base_model.output # 전이학습으로 가져온 은닉층을 저장(가져옴)

58889256/58889256 [==============================] - 3s 0us/step


In [ ]:
# 글로벌맥스풀링은 각 공간 차원이 1이 될 때까지 공간 차원에 평균 풀링을 적용하고 다른 차원은 변경하지 않고 그대로 둚.
# Flatten과 비슷한 기능을 함

output_layer = GlobalAveragePooling2D()(bm_output) # 베이스모델과 내가 작성하는 PC층 연결 => 입력값으로는 base 부분의 ouput 부분이 입력되어야 함
# 사전학습한 것과 내가 만든 풀링을 이어 붙일 때는 GloabalAveragePooling 사용
output_layer = Dense(50, activation='relu')(output_layer)
output_layer = Dense(10, activation='softmax')(output_layer)

model = Model(inputs=base_model.input, outputs=output_layer) # model을 선언할 때 inputs에는 base 부분의 input 부분이 입력되어야 함

In [ ]:
model.summary()

In [ ]:
# 최적화 함수 adam에서 학습률을 지정함으로써 과적합 방지기능 추가함

model.compile(optimizer=Adam(lr=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# 층이 깊어지면 과적합 우려 -> adam 함수에서 learning rate(학습률)를 0.001로 사용

In [ ]:
batch_size = 64
epochs = 20

history = model.fit(x=train_x, y=train_y, batch_size=batch_size, epochs=epochs, validation_data=(val_x, val_y))

Epoch 1/20
625/625 [==============================] - 46s 39ms/step - loss: 1.8791 - accuracy: 0.2511 - val_loss: 1.6034 - val_accuracy: 0.3521
Epoch 2/20
625/625 [==============================] - 24s 38ms/step - loss: 1.3793 - accuracy: 0.4618 - val_loss: 1.1858 - val_accuracy: 0.5603
Epoch 3/20
625/625 [==============================] - 23s 36ms/step - loss: 1.0671 - accuracy: 0.6120 - val_loss: 0.9939 - val_accuracy: 0.6410
Epoch 4/20
625/625 [==============================] - 24s 38ms/step - loss: 0.8527 - accuracy: 0.7047 - val_loss: 0.8593 - val_accuracy: 0.7068
Epoch 5/20
625/625 [==============================] - 24s 38ms/step - loss: 0.7292 - accuracy: 0.7523 - val_loss: 0.8405 - val_accuracy: 0.7194
Epoch 6/20
625/625 [==============================] - 24s 38ms/step - loss: 0.6108 - accuracy: 0.7973 - val_loss: 0.6951 - val_accuracy: 0.7734
Epoch 7/20
625/625 [==============================] - 23s 37ms/step - loss: 0.5328 - accuracy: 0.8244 - val_loss: 0.7351 - val_accuracy:

In [ ]:
# 테스트 데이터로 성능 평가
model.evaluate(test_images_2, test_labels_2)

313/313 [==============================] - 3s 8ms/step - loss: 0.9246 - accuracy: 0.7879


[0.9245699048042297, 0.7878999710083008]